In [1]:
import rospy
import numpy as np
import sensor_msgs.point_cloud2 as pc2
from matplotlib import pyplot as plt
from sensor_msgs.msg import PointCloud2
from jsk_recognition_msgs.msg import BoundingBoxArray,BoundingBox
from sklearn.cluster import DBSCAN
import time

def Callback(data):
    time_start = time.time()  # time start
    point_clound = pc2.read_points(data, skip_nans=True)
    d = list(point_clound)
    n_data = len(d)
    points = np.zeros((n_data, 4))
    for i in range(n_data):
        # if d[i][1]>1:
            points[i, 0] = d[i][0]
            points[i, 1] = d[i][1]
            points[i, 2] = d[i][2]
            points[i, 3] = d[i][3]
    bboxarray = BoundingBoxArray()
    bboxarray.header.frame_id = 'lidar_top'
    cluster = DBSCAN(eps=0.4, min_samples=20).fit(points[:, :3])
    labels = cluster.labels_
    unique_labels = set(labels)
    for k in zip(unique_labels):  # for every instance
        if k != -1:  # not discrete
            class_member = (labels == k)
            xyz = points[class_member]  # choose all these points
            xmax = np.max(xyz[:, 0])
            xmin = np.min(xyz[:, 0])
            ymax = np.max(xyz[:, 1])
            ymin = np.min(xyz[:, 1])
            zmax = np.max(xyz[:, 2])
            zmin = np.min(xyz[:, 2])
            width = xmax - xmin
            height = ymax - ymin
            deep = zmax - zmin
            xc = (xmax+xmin)*0.5
            yc = (ymax+ymin)* 0.5
            zc = (zmax+zmin)* 0.5
            boundingbox = BoundingBox()
            boundingbox.header.frame_id = "lidar_top"
            boundingbox.pose.orientation.w = 1
            boundingbox.pose.position.x = xc  # x boundingbox x coordinate
            boundingbox.pose.position.y = yc  # y boundingbox y coordinate
            boundingbox.pose.position.z = zc  # z boundingbox z coordinate
            boundingbox.dimensions.x = width  # width boundingbox width
            boundingbox.dimensions.y = height  # height boundingbox height
            boundingbox.dimensions.z = deep  # depth boundingbox depth
            bboxarray.boxes.append(boundingbox)
    pub_bb = rospy.Publisher('/bbox', BoundingBoxArray, queue_size=1)
    rate = rospy.Rate(20)
    pub_bb.publish(bboxarray)
    # rate.sleep()
    time_end = time.time()  # time end
    time_c = time_end - time_start  # time used
    print('time cost', time_c, 's')

def subscribe():
    rospy.init_node('bonus')
    rospy.Subscriber(name="/me5413/lidar_top",data_class=PointCloud2,callback=Callback)
    rospy.spin()

if __name__ == "__main__":
    subscribe()







ModuleNotFoundError: No module named 'rospy'